In [1]:
import sys
import os

# Determine the root directory (one level up from the notebook directory)
root_dir = os.path.abspath('..')

# Add the root directory to sys.path
sys.path.append(root_dir)

In [2]:
from web3 import AsyncWeb3, Web3
from web3.eth import Contract
from datetime import datetime


import json


def read_abi(token: str):
    with open(f"../config/{token.lower()}_abi.json") as f:
        data = json.load(f)
        return data

In [3]:
w3 = Web3(Web3.HTTPProvider("https://bitter-wandering-feather.arbitrum-mainnet.quiknode.pro/862a558ad28be94cf6b1ccae509bdca74a19086a"))

rockonyx_delta_neutral_vault_abi = read_abi("rockonyxdeltaneutralvault")
vault_contract = w3.eth.contract(
    address="0x4a10C31b642866d3A3Df2268cEcD2c5B14600523",
    abi=rockonyx_delta_neutral_vault_abi,
)


In [4]:
def get_block_by_timestamp(web3, target_timestamp, start_block, end_block):
    """Binary search to find the block closest to the target timestamp."""
    while start_block <= end_block:
        mid_block = (start_block + end_block) // 2
        block = web3.eth.get_block(mid_block)
        block_timestamp = block.timestamp

        if block_timestamp < target_timestamp:
            start_block = mid_block + 1
        elif block_timestamp > target_timestamp:
            end_block = mid_block - 1
        else:
            return mid_block  # Exact match

    # Return the closest block
    return start_block if abs(web3.eth.get_block(start_block).timestamp - target_timestamp) < abs(web3.eth.getBlock(end_block).timestamp - target_timestamp) else end_block


# Example usage

date = datetime.strptime('2024-04-11 16:05:01', '%Y-%m-%d %H:%M:%S')
target_timestamp = int(date.timestamp())  # Replace with your desired timestamp
latest_block = w3.eth.block_number

# Start searching from the genesis block to the latest block
block_number = get_block_by_timestamp(w3, target_timestamp, 0, latest_block)
block_number

199856247

In [5]:

def get_current_tvl(vault_contract: Contract, decimals = 1e6, block_number=200199390):
    tvl = vault_contract.functions.totalValueLocked().call(block_identifier=block_number)

    return tvl / decimals


get_current_tvl(vault_contract, block_number=199992141)

BadFunctionCallOutput: Could not decode contract function call to totalValueLocked with return data: b'', output_types: ['uint256']

In [8]:
def get_pps(vault_contract: Contract, decimals = 1e6, block_number=200199390):
    tvl = vault_contract.functions.pricePerShare().call(block_identifier=block_number)

    return tvl / decimals


get_pps(vault_contract, block_number=247413676)

1.02168